# Import and clean AADR data

In [1]:
#import required packages
#%pip install sklearn ipywidgets numpy pandas matplotlib requests ipywidgets IPython
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import ipywidgets as widgets
from ipywidgets import interact, Dropdown
from IPython.display import display
from sklearn import metrics
from sklearn.model_selection import train_test_split
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib notebook

In [2]:
#load version information
%load_ext version_information

In [3]:
#Version for Pandas and numpy
#Pandas (1.2.4), numpy (1.20.1), matplotlib (3.3.4), requests (2.25.1), json (2.0.9) were used for this assignment
%version_information pandas, numpy, matplotlib, requests, json, ipywidgets, IPython, sklearn, scipy

Software,Version
Python,3.8.8 64bit [MSC v.1916 64 bit (AMD64)]
IPython,7.22.0
OS,Windows 10 10.0.22000 SP0
pandas,1.2.4
numpy,1.20.1
matplotlib,3.3.4
requests,2.25.1
json,2.0.9
ipywidgets,7.6.3
IPython,7.22.0


In [27]:
#list of functions

#function to get API data
def API_ping (API):
    api_response = requests.get(API)
    if api_response:
        print('request successful')
    else:
        print('request not successful')
        print(api_response.status_code)

#get charcteristics of a dataframe
def df_char(df):
    print('Shape of dataframe:', df.shape)
    print('\nColumn names:',df.columns, '\n')
    print( df.info())

#clean column names
def clean_columns(df):
    df.columns = df.columns.str.strip(" ")
    df.columns = df.columns.str.strip("_")
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.replace('-', '_')
    df.columns = df.columns.str.replace('113_cause_name', 'cause_name_113')
    df.columns = df.columns.str.replace('Age_adjusted_Rate', 'AADR')
    df.columns = df.columns.str.replace('Number_of_Deaths', 'Deaths')
    df.columns = df.columns.str.replace('Abbreviation', 'Ab')
    print(df.columns)


#graph two data sets
def graph_me(df1, df2, PTitle, Label1, Label2):
    fig1 = plt.figure(1)
    plt. clf()
    plt.plot(df1.year,df1.population, label= Label1)
    plt.plot(df2.year,df2.population,label= Label2)
    plt.title(PTitle)
    plt.ylabel(df1.columns[1])
    plt.xlabel(df1.columns[0])
    #plt.xticks(ticks=[2000,2005,2010,2015])
    plt.legend()
    plt.show()
    
#graph two data sets for figure 3
def graph_me2(df1, df2, PTitle, Label1, Label2):
    df1 = df1.diff()
    df2 = df2.diff()
    fig3 = plt.figure(3)
    plt. clf()
    plt.plot(df1.year,df1.population, label= Label1)
    plt.plot(df2.year,df2.population,label= Label2)
    plt.title(PTitle)
    plt.ylabel(df1.columns[1])
    plt.xlabel(df1.columns[0])
    #plt.xticks(ticks=[2000,2005,2010,2015])
    plt.legend()
    plt.show()

#predict population based on area and year
#Most of the below is adapted from 
#https://medium.com/codex/stock-predication-using-regression-algorithm-in-python-fb8b426453b9
def linear_regression(df):
    x = df[['area', 'year']]
    y = df['population']
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.15 , shuffle=False,random_state = 0)
    regression = LinearRegression()
    regression.fit(train_x, train_y)
    print('\n', "regression coefficient",regression.coef_)
    print('\n', "regression intercept",regression.intercept_)
    regression_confidence = regression.score(test_x, test_y)
    print('\n', "linear regression confidence: ", regression_confidence)
    predicted = regression.predict(test_x)
    dflr=pd.DataFrame({'Actual_Population':test_y, 'Predicted_Population':predicted})
    print(dflr.head(10))
    print(dflr.tail(10))
    print('\n', 'Mean Absolute Error (MAE):', metrics.mean_absolute_error(test_y, predicted))
    print('Mean Squared Error (MSE) :', metrics.mean_squared_error(test_y, predicted))
    print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(test_y, predicted)))
    x2 = dflr.Actual_Population.mean()
    y2 = dflr.Predicted_Population.mean()
    Accuracy1 = x2/y2*100
    print('\n', "The accuracy of the model is " , Accuracy1)
    #fig3 = plt.figure(3)
    #plt.scatter(dflr.Actual_Population, dflr.Predicted_Population,  color='Darkblue')
    #plt.xlabel("Actual Population")
    #plt.ylabel("Predicted Population")
    #plt.show()
    slope, intercept, r_value, p_value, std_err = linregress(dflr)
    #fig2 = plt.figure(2)
    #plt.plot(dflr.Actual_Population, color='black',label='Acutal')
    #plt.plot(dflr.Predicted_Population, color='lightblue',label='Predicted')
    #plt.plot((intercept + slope*dflr.Predicted_Population), 'r', label='fitted line')
    #plt.title("Population predicted")
    #plt.legend();
    #plt.show()
    prediction_population = []
    prediction_year = []
    for i in range(50):
        #print(i+1990)
        temp_pop = regression.coef_[1]*(i+1990) + regression.intercept_
        prediction_population.append(temp_pop)
        prediction_year.append(i+1990)
        #print(prediction_population)
        #print(prediction_year)
    df_Prediction = pd.DataFrame(prediction_population,prediction_year)
    #print(df_Prediction)
    fig2 = plt.figure(2)
    plt.plot(df_Prediction, color = 'black',label='predicted')
    plt.plot(df.year,df.population, color = 'blue', label = 'actual')
    plt.title("Population predicted")
    plt.legend()
    plt.xlabel('year')
    plt.ylabel('population')
    plt.show()
    
    return dflr

In [23]:
#import data
Overdose_Deaths = 'https://raw.githubusercontent.com/boward10/Opioid_Overdose_Dashboard_Project/main/CDC%20Injury%20Center%20Drug%20Overdose%20Deaths.csv'
print(Overdose_Deaths)
AADR = pd.read_csv(Overdose_Deaths)
print(AADR.head())

https://raw.githubusercontent.com/boward10/Opioid_Overdose_Dashboard_Project/main/CDC%20Injury%20Center%20Drug%20Overdose%20Deaths.csv
        State State Abbreviation  \
0     Alabama                 AL   
1      Alaska                 AK   
2     Arizona                 AZ   
3    Arkansas                 AR   
4  California                 CA   

   2019 Age-adjusted Rate (per 100,000 population)  2019 Number of Deaths  \
0                                             16.3                    768   
1                                             17.8                    132   
2                                             26.8                   1907   
3                                             13.5                    388   
4                                             15.0                   6198   

   2018 Age-adjusted Rate (per 100,000 population)  2018 Number of Deaths  \
0                                             16.6                    775   
1                              

In [28]:
#clean up variable names
clean_columns(AADR)
print(AADR.columns)

Index(['State', 'State_Ab', '2019_AADR_(per_100,000_population)',
       '2019_Deaths', '2018_AADR_(per_100,000_population)', '2018_Deaths',
       '2017_AADR_(per_100,000_population)', '2017_Deaths',
       '2016_AADR_(per_100,000_population)', '2016_Deaths',
       '2015_AADR_(per_100,000_population)', '2015_Deaths',
       '2014_AADR_(per_100,000_population)', '2014_Deaths',
       '2013_AADR_(per_100,000_population)', '2013_Deaths',
       '2019_Poverty_rate_(percent_of_persons_in_poverty)',
       'Gini_coefficient_of_income_inequality', 'GDP_per_capita_2021',
       'GDP_(nominal_in_millions_of_USD)_2021',
       'Urban_population_as_a_percentage_of_the_total_population_in_2010',
       'Population_density_per_km2', 'Population', 'Land_Area_(km2)'],
      dtype='object')
Index(['State', 'State_Ab', '2019_AADR_(per_100,000_population)',
       '2019_Deaths', '2018_AADR_(per_100,000_population)', '2018_Deaths',
       '2017_AADR_(per_100,000_population)', '2017_Deaths',
       '201

In [30]:
#check dataframe
df_char(AADR)

Shape of dataframe: (51, 24)

Column names: Index(['State', 'State_Ab', '2019_AADR_(per_100,000_population)',
       '2019_Deaths', '2018_AADR_(per_100,000_population)', '2018_Deaths',
       '2017_AADR_(per_100,000_population)', '2017_Deaths',
       '2016_AADR_(per_100,000_population)', '2016_Deaths',
       '2015_AADR_(per_100,000_population)', '2015_Deaths',
       '2014_AADR_(per_100,000_population)', '2014_Deaths',
       '2013_AADR_(per_100,000_population)', '2013_Deaths',
       '2019_Poverty_rate_(percent_of_persons_in_poverty)',
       'Gini_coefficient_of_income_inequality', 'GDP_per_capita_2021',
       'GDP_(nominal_in_millions_of_USD)_2021',
       'Urban_population_as_a_percentage_of_the_total_population_in_2010',
       'Population_density_per_km2', 'Population', 'Land_Area_(km2)'],
      dtype='object') 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 24 columns):
 #   Column                                                    

AttributeError: 'DataFrame' object has no attribute 'null'

In [31]:
#Check for nulls
AADR.isnull()

,State,State_Ab,"2019_AADR_(per_100,000_population)",2019_Deaths,"2018_AADR_(per_100,000_population)",2018_Deaths,"2017_AADR_(per_100,000_population)",2017_Deaths,"2016_AADR_(per_100,000_population)",2016_Deaths,...,"2013_AADR_(per_100,000_population)",2013_Deaths,2019_Poverty_rate_(percent_of_persons_in_poverty),Gini_coefficient_of_income_inequality,GDP_per_capita_2021,GDP_(nominal_in_millions_of_USD)_2021,Urban_population_as_a_percentage_of_the_total_population_in_2010,Population_density_per_km2,Population,Land_Area_(km2)
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [35]:
#nulls are in DC
AADR.State

0                  Alabama
1                   Alaska
2                  Arizona
3                 Arkansas
4               California
5                 Colorado
6              Connecticut
7                 Delaware
8     District of Columbia
9                  Florida
10                 Georgia
11                  Hawaii
12                   Idaho
13                Illinois
14                 Indiana
15                    Iowa
16                  Kansas
17                Kentucky
18               Louisiana
19                   Maine
20                Maryland
21           Massachusetts
22                Michigan
23               Minnesota
24             Mississippi
25                Missouri
26                 Montana
27                Nebraska
28                  Nevada
29           New Hampshire
30              New Jersey
31              New Mexico
32                New York
33          North Carolina
34            North Dakota
35                    Ohio
36                Oklahoma
3

In [40]:
AADR_2 = AADR[AADR.State != 'District of Columbia']
df_char(AADR_2)

Shape of dataframe: (50, 24)

Column names: Index(['State', 'State_Ab', '2019_AADR_(per_100,000_population)',
       '2019_Deaths', '2018_AADR_(per_100,000_population)', '2018_Deaths',
       '2017_AADR_(per_100,000_population)', '2017_Deaths',
       '2016_AADR_(per_100,000_population)', '2016_Deaths',
       '2015_AADR_(per_100,000_population)', '2015_Deaths',
       '2014_AADR_(per_100,000_population)', '2014_Deaths',
       '2013_AADR_(per_100,000_population)', '2013_Deaths',
       '2019_Poverty_rate_(percent_of_persons_in_poverty)',
       'Gini_coefficient_of_income_inequality', 'GDP_per_capita_2021',
       'GDP_(nominal_in_millions_of_USD)_2021',
       'Urban_population_as_a_percentage_of_the_total_population_in_2010',
       'Population_density_per_km2', 'Population', 'Land_Area_(km2)'],
      dtype='object') 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 50
Data columns (total 24 columns):
 #   Column                                                    